In [1]:
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_file = open("datasets/stanford-covid-vaccine/train.json", "r")
train_data_raw = train_file.read()
train_data = list(map(lambda l: json.loads(l), list(filter(lambda l: len(l) > 0, train_data_raw.split("\n")))))
train_file.close()

test_file = open("datasets/stanford-covid-vaccine/test.json", "r")
test_data_raw = test_file.read()
test_data = list(map(lambda l: json.loads(l), list(filter(lambda l: len(l) > 0, test_data_raw.split("\n")))))
test_file.close()

In [3]:
train_data[0].keys()

dict_keys(['index', 'id', 'sequence', 'structure', 'predicted_loop_type', 'signal_to_noise', 'SN_filter', 'seq_length', 'seq_scored', 'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C'])

In [4]:
train_data[0]["sequence"]

'GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC'

In [5]:
train_data[0]["structure"]

'.....((((((.......)))).)).((.....((..((((((....))))))..)).....))....(((((((....))))))).....................'

In [6]:
train_data[0]["predicted_loop_type"]

'EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHHHSSSSSSIISSIIIIISSXXXXSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE'

In [7]:
train_data[0]["id"], len(train_data[0]["sequence"]), len(train_data[0]["structure"]), len(train_data[0]["predicted_loop_type"])

('id_001f94081', 107, 107, 107)

In [8]:
input_sequence = list(zip(train_data[0]["sequence"], train_data[0]["structure"], train_data[0]["predicted_loop_type"]))

In [9]:
len(input_sequence)

107

In [10]:
input_sequence[0], input_sequence[-1]

(('G', '.', 'E'), ('C', '.', 'E'))

In [11]:
train_data[0]["reactivity"][0], train_data[0]["deg_Mg_pH10"][0], train_data[0]["deg_pH10"][0], train_data[0]["deg_Mg_50C"][0], train_data[0]["deg_50C"][0] 

(0.3297, 0.7556, 2.3375, 0.3581, 0.6382)

In [12]:
output_sequence = list(zip(train_data[0]["reactivity"], train_data[0]["deg_Mg_pH10"], train_data[0]["deg_pH10"], train_data[0]["deg_Mg_50C"], train_data[0]["deg_50C"]))

In [13]:
len(output_sequence)

68

In [14]:
output_sequence[0], output_sequence[-1]

((0.3297, 0.7556, 2.3375, 0.3581, 0.6382),
 (0.5731, 0.6898, 1.172, 0.4254, 0.8472))

In [15]:
def build_categorical_encoder(input_attribute_index_or_key, categories, total_dimensions, start_index, end_index):
    if not (end_index - start_index + 1 == len(categories)):
        raise Exception("Mismatch between number of categories and dimensions assigned.")
    def encoder(data_row, data_vector):
        if len(data_vector) != total_dimensions:
            raise Exception(f"Data vector is of size {len(data_vector)}, but should be of size {total_dimensions}.")
        
        category_index = categories.index(data_row[input_attribute_index_or_key])
        encoding_index = start_index + category_index
        data_vector[encoding_index] = 1
    
    return encoder

def build_continuous_encoder(input_attribute_index_or_key, total_dimensions, attribute_index):
    def encoder(data_row, data_vector):
        if len(data_vector) != total_dimensions:
            raise Exception(f"Data vector is of size {len(data_vector)}, but should be of size {total_dimensions}.")
        data_vector[attribute_index] = float(data_row[input_attribute_index_or_key])
    
    return encoder

def encode_data(rows, encoders, total_dimensions):
    encoded_rows = []
    for row in rows:
        encoded_row = [0] * total_dimensions
        for encoder in encoders:
            encoder(row, encoded_row)
        encoded_rows.append(encoded_row)
    
    return encoded_rows

In [16]:
total_dim_input = 14
total_dim_output = 5
sequence_classes = ["A", "G", "U", "C"]
structure_classes = ["(", ".", ")"]
predicted_loop_type_classes = ["S", "M", "I", "B", "H", "E", "X"]

sequence_encoder = build_categorical_encoder(0, sequence_classes, total_dim_input, 0, 3)
structure_encoder = build_categorical_encoder(1, structure_classes, total_dim_input, 4, 6)
predicted_loop_encoder = build_categorical_encoder(2, predicted_loop_type_classes, total_dim_input, 7, 13)


data_encoders = [
    sequence_encoder,
    structure_encoder,
    predicted_loop_encoder
]

In [17]:
encoded_input_sequence = encode_data(input_sequence, data_encoders, total_dim_input)

In [18]:
input_sequence[5]

('A', '(', 'S')

In [19]:
encoded_input_sequence[5]

[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [20]:
x = torch.tensor(encoded_input_sequence, dtype=torch.float32)

In [21]:
x.size()

torch.Size([107, 14])

In [22]:
filtered_train_data = list(filter(lambda d: d["SN_filter"] == 1, train_data))
dataset = []
for row in filtered_train_data:
    input_sequence = list(zip(row["sequence"], row["structure"], row["predicted_loop_type"]))
    target_sequence = list(zip(row["reactivity"], row["deg_Mg_pH10"], row["deg_pH10"], row["deg_Mg_50C"], row["deg_50C"]))
    input_sequence_encoded = encode_data(input_sequence, data_encoders, total_dim_input)
    dataset.append(
        (input_sequence_encoded,
         target_sequence))


testing_sequence_id_to_dataset_map = dict()
testing_set = []
for row in test_data:
    input_sequence = list(zip(row["sequence"], row["structure"], row["predicted_loop_type"]))
    input_sequence_encoded = encode_data(input_sequence, data_encoders, total_dim_input)
    testing_set.append(input_sequence_encoded)
    testing_sequence_id_to_dataset_map[row["id"]] = input_sequence_encoded


In [23]:
random.shuffle(dataset)

training = dataset[:int(len(dataset) * 0.8)]
evaluation = dataset[int(len(dataset) * 0.8):]

In [24]:
len(training), len(evaluation), len(testing_set)

(1271, 318, 3634)

In [25]:
num_epochs = 100
batch_size_train = 31
batch_size_test = 318

In [26]:
def pad(seq, length, padding=None):
    if padding == None:
        padding = 0
    z = [padding] * (length - len(seq))
    seq = seq + z
    
    return seq

In [27]:
# class RNNModelRNA(nn.Module):
    
#     def __init__(self, input_dim=None, hidden_dim=None, num_layers=None, output_dim=None):
#         super(RNNModelRNA, self).__init__()
#         self.num_layers = num_layers
#         self.hidden_dim = hidden_dim
#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim
#         self.rnn = nn.RNN(input_dim, hidden_dim, num_layers)
#         self.fc = nn.Linear(hidden_dim, output_dim)
    
#     def init_hidden(self, batch_size):
#         return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to("cuda")
    
#     def forward(self, x, lengths=None):
#         batch_size = x.size(1)
#         hidden = self.init_hidden(batch_size)
#         if lengths is not None:
#             x = nn.utils.rnn.pack_padded_sequence(x, lengths)
#             output, hidden = self.rnn(x, hidden)
#             output, lengths_unpacked = nn.utils.rnn.pad_packed_sequence(output)
#         else:
#             output, hidden = self.rnn(x, hidden)

#         output = self.fc(output)

#         return output

    
# class LSTMModelRNA(nn.Module):
    
#     def __init__(self, input_dim=None, hidden_dim=None, num_layers=None, output_dim=None, dropout=0.1):
#         super(LSTMModelRNA, self).__init__()
#         self.num_layers = num_layers
#         self.hidden_dim = hidden_dim
#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim
#         self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout, bidirectional=True)
#         self.fc = nn.Linear(2*hidden_dim, output_dim)
    
#     def init_hidden(self, batch_size):
#         return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to("cuda")
    
#     def forward(self, x, lengths=None):
#         batch_size = x.size(1)
#         hidden = self.init_hidden(batch_size)
#         if lengths is not None:
#             x = nn.utils.rnn.pack_padded_sequence(x, lengths)
#             output, _ = self.rnn(x)
#             output, lengths_unpacked = nn.utils.rnn.pad_packed_sequence(output)
#         else:
#             output, _ = self.rnn(x)

#         output = self.fc(output)

#         return output

In [28]:
# # rnn = RNNModelRNA(input_dim=14, hidden_dim=20, num_layers=3, output_dim=5).cuda()
# rnn = LSTMModelRNA(input_dim=14, hidden_dim=20, num_layers=5, output_dim=5, dropout=0.2).cuda()
# hidden = torch.zeros(1, 1, 3).to("cuda")
# optimizer = optim.Adam(rnn.parameters(), lr=0.001)

In [29]:
# num_epochs = 100
# batch_size_train = 480
# batch_size_test = 480

padding = (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
train_batched = []
prev = 0
for i in range(batch_size_train, len(training) + 1, batch_size_train):
    batch = training[prev:i]
    batch = sorted(batch, key=lambda r: len(r[0]), reverse=True)
    lengths_batch = list(map(lambda r: len(r[0]), batch))
    max_length = max(lengths_batch)
    batch_padded = list(map(lambda s: pad(s[0], max_length, padding=padding), batch))
    batch_targets = list(map(lambda r: r[1], batch))
    train_batched.append(((batch_padded, lengths_batch), batch_targets))
    prev = i

    
evaluation_batched = []
prev = 0
for i in range(batch_size_test, len(evaluation) + 1, batch_size_test):
    batch = evaluation[prev:i]
    batch = sorted(batch, key=lambda r: len(r[0]), reverse=True)
    lengths_batch = list(map(lambda r: len(r[0]), batch))
    max_length = max(lengths_batch)
    batch_padded = list(map(lambda s: pad(s[0], max_length, padding=padding), batch))
    batch_targets = list(map(lambda r: r[1], batch))
    evaluation_batched.append(((batch_padded, lengths_batch), batch_targets))
    prev = i

In [30]:
len(train_batched), len(evaluation_batched)

(41, 1)

In [31]:
len(train_batched[0][0][0]), len(evaluation_batched[0][0][0])

(31, 318)

In [32]:
def batch_to_tensor(data_batch, lengths, batchsize):
    batch_tensors = list(map(lambda t: torch.tensor(t, dtype=torch.float32), data_batch))
    return torch.stack(batch_tensors, dim=1)

In [33]:
training_set_batches = []
evaluation_set_batches = []
for ((data_batch, batch_lengths), target) in train_batched:
    training_set_batches.append(((batch_to_tensor(data_batch, batch_lengths, batch_size_train), batch_lengths), target))

for ((data_batch, batch_lengths), target) in evaluation_batched:
    evaluation_set_batches.append(((batch_to_tensor(data_batch, batch_lengths, batch_size_test), batch_lengths), target))

In [34]:
training_set_batch = training_set_batches[0]
((data_batch, lengths_batch), target) = training_set_batch
# output = rnn(data_batch, lengths=training)

In [35]:
class RNNModelRNA(nn.Module):
    
    def __init__(self, input_dim=None, hidden_dim=None, num_layers=None, output_dim=None):
        super(RNNModelRNA, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to("cuda")
    
    def forward(self, x, lengths=None):
        batch_size = x.size(1)
        hidden = self.init_hidden(batch_size)
        if lengths is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, lengths)
            output, hidden = self.rnn(x, hidden)
            output, lengths_unpacked = nn.utils.rnn.pad_packed_sequence(output)
        else:
            output, hidden = self.rnn(x, hidden)

        output = self.fc(output)

        return output

    
class LSTMModelRNA(nn.Module):
    
    def __init__(self, input_dim=None, hidden_dim=None, num_layers=None, output_dim=None, dropout=0.1):
        super(LSTMModelRNA, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(2*hidden_dim, output_dim)
    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to("cuda")
    
    def forward(self, x, lengths=None):
        batch_size = x.size(1)
        hidden = self.init_hidden(batch_size)
        if lengths is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, lengths)
            output, _ = self.rnn(x)
            output, lengths_unpacked = nn.utils.rnn.pad_packed_sequence(output)
        else:
            output, _ = self.rnn(x)

        output = self.fc(output)

        return output

In [55]:
# rnn = RNNModelRNA(input_dim=14, hidden_dim=20, num_layers=3, output_dim=5).cuda()
rnn = LSTMModelRNA(input_dim=14, hidden_dim=20, num_layers=9, output_dim=5, dropout=0.2).cuda()
hidden = torch.zeros(1, 1, 3).to("cuda")
optimizer = optim.Adam(rnn.parameters(), lr=0.01)

In [56]:
output = rnn(data_batch.to("cuda"), lengths=lengths_batch)

In [57]:
output[:,0].size()

torch.Size([107, 5])

In [58]:
def test():
    with torch.no_grad():
        test_loss = torch.tensor(0.0).to("cuda")
        for ((data_batch, lengths_batch), target) in evaluation_set_batches:
            output = rnn(data_batch.to("cuda"), lengths=lengths_batch)
            loss_sequence = torch.tensor(0.0).to("cuda")
            for i in range(batch_size_test):
                loss_sequence += torch.nn.functional.mse_loss(output[:len(target[i]),i,:], torch.tensor(target[i]).to("cuda"))
            
            test_loss += (loss_sequence / batch_size_test)
        
        return test_loss

In [ ]:
train_losses = []
test_losses = []
train_counter = range(num_epochs * len(training_set_batches))
test_counter = range(num_epochs * len(training_set_batches))
for n in range(num_epochs):
    for ((data_batch, lengths_batch), target) in training_set_batches:
        optimizer.zero_grad()
        rnn.zero_grad()
        output = rnn(data_batch.to("cuda"), lengths=lengths_batch)
        loss = torch.tensor(0.0).to("cuda")
        sequence_loss = torch.tensor(0.0).to("cuda")
        for i in range(batch_size_train):
            sequence_loss += torch.nn.functional.mse_loss(output[:len(target[i]),i,:], torch.tensor(target[i]).to("cuda"))
        loss += (sequence_loss / batch_size_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_losses.append(torch.sqrt(loss).item())
        test_loss = test()
        test_losses.append(torch.sqrt(test_loss).item())

In [ ]:
fig, ax = plt.subplots()
ax.plot(train_counter, train_losses, color="b", label="Train Loss")
ax.scatter(test_counter, test_losses, color="red", label="Test Loss")
leg = ax.legend()

In [ ]:
train_losses[-1], test_losses[-1]

In [43]:
list(testing_sequence_id_to_dataset_map.keys())[0]

'id_00073f8be'

In [44]:
input_seq = testing_sequence_id_to_dataset_map["id_00073f8be"]

In [45]:
rnn.eval()

LSTMModelRNA(
  (rnn): LSTM(14, 20, num_layers=9, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=40, out_features=5, bias=True)
)

In [46]:
input_seq = torch.tensor(input_seq, dtype=torch.float32)

In [47]:
input_seq = input_seq.view(107, 1, 14)

In [48]:
output_seq = rnn(input_seq.to("cuda"))

In [49]:
output_seq.size()

torch.Size([107, 1, 5])

In [50]:
output_seq[0, 0, :]

tensor([0.7153, 0.7475, 2.1112, 0.6229, 0.7481], device='cuda:0',
       grad_fn=<SliceBackward>)

In [51]:
output_header = 'id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C'
output_lines = [output_header]
for sequence_id, test_seq in testing_sequence_id_to_dataset_map.items():
    test_seq = torch.tensor(test_seq, dtype=torch.float32)
    test_seq = test_seq.view(test_seq.size(0), 1, test_seq.size(1))
    output = rnn(test_seq.to("cuda"))
    output = output.to("cpu")
    seq_length = output.size(0)
    for i in range(seq_length):
        seq_id_pos = f"{sequence_id}_{i}"
        seq_id_pos_entry = list(map(lambda num: str(num), output[i,0,:].tolist()))
        submission_line_entries = [seq_id_pos] + seq_id_pos_entry
        submission_line = ",".join(submission_line_entries)
        output_lines.append(submission_line)

In [52]:
submission_data = "\n".join(output_lines)

In [53]:
output_lines[1]

'id_00073f8be_0,0.7152865529060364,0.7475160360336304,2.1112401485443115,0.6229220628738403,0.7481404542922974'

In [54]:
submission_file = open("datasets/stanford-covid-vaccine/submission-lstm.csv", "w")
submission_file.write(submission_data)
submission_file.close()